## Marvin
> "Let’s build robots with Genuine People Personalities," they said. So they tried it out with me. I’m a personality prototype. You can tell, can’t you?

## Note:
- Marvin is purely for internal use
- Marvin is unrelated to the Prefect slack app that customers will interact with

## What we want to talk about today:
- what makes Marvin tick?
- what do you need locally to hack on Marvin?

<img src="marvins_guts.svg">

#### Marvin's high-level structure


```
|____marvin
| |____defcon.py      # /defcon command
| |____responses.py   # notifications / reactions
| |______init__.py
| |____core.py        # webserver
| |____loop_policy.py # utility for scheduling standup
| |____standup.py     # standup logic
| |____utilities.py   # slack utilities

## deployment
|____kubernetes
| |____deployment.yml
| |____service.yml

|____.circleci
| |____config.yml
```

## And unfortunately, there's a relatively large portion of work that needs to occur in the Slack Web UI
<img src="slack_web_ui.png">

# Tokens

| Token Name        | Purpose           |
| ------------- |:-------------:| -----:|
| `MARVIN_SLACK_TOKEN`      | Marvin's "bot user" token; this is the token Marvin needs for simply speaking / reacting |
| `MARVIN_SLACK_OAUTH_TOKEN`      | Marvin's full OAuth token; this is the token Marvin needs for performing higher permission actions (such as editing pins) |
| `MARVIN_SLACK_VALIDATION_TOKEN` | this token is used to internally validate that incoming requests are legitimate Slack requests |
| `GOOGLE_SERVICE_ACCOUNT_KEY` | Marvin's key granting permissions for Googe Cloud services |

# Things you can do locally with _only_ these tokens:

- Speak through Marvin:

```python

from marvin.utilities import say
say("Hey this is really Chris", channel="CBH18KG8G") # engineering channel
```

- edit Marvin's code, run tests (of course there are unit tests!)
- deploy Marvin from your laptop
- anything that doesn't require a running webserver

# Marvin's webserver

Slash commands and event subscriptions are sent to a fixed web address, which normally points to a Google Cloud IP Address; you can easily change this to point to a local (and temporary) address.  Run the `marvin` CLI command + `ngrok http 8080` to standup a webserver and
<img src="ngrok_slack.png" width=500>

Notice the error message: this is the `MARVIN_SLACK_VALIDATION_TOKEN`

# How Notifications Work: Simple

1. a Slack channel exists which each notification stream is posted to (e.g., #github-notifications)
2. Marvin is invited to that channel
3. Using Marvin's event subscription to channel posts, he listens for each particular bot and forwards the notification to the right person:

In [ ]:
async def event_handler(data: Body):
    ...
    json_data = json.loads(data)                                                    
    ...
    event = json_data.get("event", {})                                              
    event_type = event.get("type")                                                  
    if event_type == "app_mention" or MARVIN_ID in event.get("text", ""):
        return app_mention(event)
    elif event_type == "message" and event.get("bot_id") == "BBGMPFDHQ":
        return github_mention(event)
    elif event_type == "message" and event.get("bot_id") == "BDUBG9WAD":
        return notion_mention(event)

## Source code for Github mentions

In [ ]:
def github_mention(event):                                                       
    data = event.get("attachments", [{}])[0]                                     
    text = data.get("text", "")                                                  
    was_mentioned = {uid: (f"@{github}" in text) for github, uid in GITHUB_MAP.items()}
    for uid, mentioned in was_mentioned.items():                                 
        if not mentioned:                                                        
            continue                                                             
        else:                                                                    
            link = data.get("title_link", "link unavailable")                    
            msg = f"You were mentioned on GitHub @ {link}"                       
            say(msg, channel=get_dm_channel_id(uid))

# How do you figure out channel IDs and user IDs??

In [ ]:
from marvin.utilities import get_users

get_users?
>>> Retrieve all current full-time Slack users.

>>> Returns:
>>>    - a dictionary of user name -> Slack User ID

get_users()
# {'jeremiah': 'UAPLR5SHL',
#  'chris': 'UBBE1SC8L',
#  'josh': 'UBE4N2LG1',
#  'dylan': 'UDKF9U8UC',
#  'ngan': 'UDTREHXGD'}

## and similarly for get_channels

## How does Marvin remember things??

#### Defcon Levels
Marvin doesn't. He simply searchs for a pinned item of the right form.

#### Standup
Marvin maintains a global dictionary of updates - this means if the process running Marvin is killed, he will lose any updates he has received since the last standup.  **Marvin needs a memory!**

# The Future

<img src="marvin_plus_prefect.png">

## Marvin will run on Prefect!